# Post-Rental Failed Charge Model - Decision Tree

### Targets which rental week will have failed charges in the next 45 days

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime, timedelta
import time as systime
from lyft_analysis.db import presto
import math

In [2]:

sql = """


WITH 

  rentals AS (

  SELECT lyft_id,
         region,
         rental_id,
         rental_provider_id,
         start_date_time,
         end_date_time
    FROM hive.redshift.dimension_rentals
   WHERE end_date_time < timestamp '2019-04-01'
     AND end_date_time > timestamp '2018-06-01'
     AND start_date_time + interval '90' day > end_date_time

)

, failed_charges_and_chargebacks AS (

  SELECT r.lyft_id,
         r.region,
         r.rental_id,
         r.rental_provider_id,
         r.start_date_time,
         r.end_date_time,
         
         --FAILED CHARGES
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) + interval '8' day AND DATE_TRUNC('day',r.end_date_time) + interval '45' day THEN 1 ELSE 0 END) AS num_failed_charges_45d_post_rental,
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) + interval '8' day AND DATE_TRUNC('day',r.end_date_time) + interval '45' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_failed_charges_45d_post_rental,
                  
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) < DATE_TRUNC('day',r.end_date_time) THEN 1 ELSE 0 END) AS num_failed_charges_pre_rental,
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '21' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_failed_charges_prior_3wks,
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '14' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_failed_charges_prior_2wks,
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '7' day  AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_failed_charges_prior_1wk,
         
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) < DATE_TRUNC('day',r.end_date_time) THEN fgc.amount ELSE 0 END)*0.01 AS sum_failed_charges_pre_rental,
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '21' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_failed_charges_prior_3wks,
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '14' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_failed_charges_prior_2wks,
         SUM(CASE WHEN fgc.status = 'failed' AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '7' day  AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_failed_charges_prior_1wk,
         
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) < DATE_TRUNC('day',r.end_date_time) THEN 1 ELSE 0 END) AS num_init_failed_charges_pre_rental,
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '21' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_init_failed_charges_prior_3wks,
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '14' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_init_failed_charges_prior_2wks,
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '7' day  AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_init_failed_charges_prior_1wk,
         
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) < DATE_TRUNC('day',r.end_date_time) THEN fgc.amount ELSE 0 END)*0.01 AS sum_init_failed_charges_pre_rental,
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '21' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_init_failed_charges_prior_3wks,
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '14' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_init_failed_charges_prior_2wks,
         SUM(CASE WHEN (DATE_TRUNC('day',fgc.created_at) <> DATE_TRUNC('day',fgc.succeeded_at) OR fgc.status = 'failed') AND DATE_TRUNC('day',fgc.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '7' day  AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_init_failed_charges_prior_1wk,
         
         --CHARGEBACKS
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) + interval '8' day AND DATE_TRUNC('day',r.end_date_time) + interval '45' day THEN 1 ELSE 0 END) AS num_chargebacks_45d_post_rental,
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) + interval '8' day AND DATE_TRUNC('day',r.end_date_time) + interval '45' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_chargebacks_45d_post_rental,

         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) < DATE_TRUNC('day',r.end_date_time) THEN 1 ELSE 0 END) AS num_chargebacks_pre_rental,
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '21' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_chargebacks_prior_3wks,
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '14' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_chargebacks_prior_2wks,
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '7' day  AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN 1 ELSE 0 END) AS num_chargebacks_prior_1wk,
         
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) < DATE_TRUNC('day',r.end_date_time) THEN fgc.amount ELSE 0 END)*0.01 AS sum_chargebacks_pre_rental,
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '21' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_chargebacks_prior_3wks,
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '14' day AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_chargebacks_prior_2wks,
         SUM(CASE WHEN cb.charge_id IS NOT NULL AND DATE_TRUNC('day',cb.created_at) BETWEEN DATE_TRUNC('day',r.end_date_time) - interval '7' day  AND DATE_TRUNC('day',r.end_date_time) - interval '1' day THEN fgc.amount ELSE 0 END)*0.01 AS sum_chargebacks_prior_1wk
         
    FROM rentals r
    LEFT JOIN hive.redshift.fact_green_charges fgc
      ON r.lyft_id = fgc.lyft_id
     AND fgc.order_type IN('rental_deposit','rentalSurcharge')
     AND fgc.status <> 'forgiven'
    LEFT JOIN hive.redshift.fact_green_charge_adjustments cb
      ON cb.charge_id = fgc.id
     AND cb.type IN('disputeCreated')
   GROUP BY 1,2,3,4,5,6
  
)

, driving_behavior AS (

  SELECT fcac.*,
         
         --RIDES
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) + interval '8' day AND DATE_TRUNC('day',fcac.end_date_time) + interval '45' day THEN p.rides_completed ELSE 0 END) AS rides_45d_post_rental,
         SUM(CASE WHEN p.time_id < DATE_TRUNC('day',fcac.end_date_time)                                                  THEN p.rides_completed ELSE 0 END) AS rides_pre_rental,
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval '21' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day THEN p.rides_completed ELSE 0 END) AS rides_prior_3wks,
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval '14' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day THEN p.rides_completed ELSE 0 END) AS rides_prior_2wks,
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval  '7' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day THEN p.rides_completed ELSE 0 END) AS rides_prior_1wks,

         --HOURS
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) + interval '8' day AND DATE_TRUNC('day',fcac.end_date_time) + interval '45' day THEN p.hours_worked ELSE 0 END) AS hours_45d_post_rental,
         SUM(CASE WHEN p.time_id < DATE_TRUNC('day',fcac.end_date_time)                                                                                       THEN p.hours_worked ELSE 0 END) AS hours_pre_rental,
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval '21' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day THEN p.hours_worked ELSE 0 END) AS hours_prior_3wks,
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval '14' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day THEN p.hours_worked ELSE 0 END) AS hours_prior_2wks,
         SUM(CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval  '7' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day THEN p.hours_worked ELSE 0 END) AS hours_prior_1wks,

         --DAYS ACTIVE
         COUNT(DISTINCT CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) + interval '8' day AND DATE_TRUNC('day',fcac.end_date_time) + interval '45' day AND p.rides_completed > 1 THEN p.time_id ELSE NULL END) AS days_active_45d_post_rental,
         COUNT(DISTINCT CASE WHEN p.time_id < DATE_TRUNC('day',fcac.end_date_time)                                                                                       AND p.rides_completed > 1 THEN p.time_id ELSE NULL END) AS days_active_pre_rental,
         COUNT(DISTINCT CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval '21' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day AND p.rides_completed > 1 THEN p.time_id ELSE NULL END) AS days_active_prior_3wks,
         COUNT(DISTINCT CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval '14' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day AND p.rides_completed > 1 THEN p.time_id ELSE NULL END) AS days_active_prior_2wks,
         COUNT(DISTINCT CASE WHEN p.time_id BETWEEN DATE_TRUNC('day',fcac.end_date_time) - interval  '7' day AND DATE_TRUNC('day',fcac.end_date_time) - interval '1' day AND p.rides_completed > 1 THEN p.time_id ELSE NULL END) AS days_active_prior_1wks
         
    FROM failed_charges_and_chargebacks fcac
    LEFT JOIN hive.core.rollup_driver_region_periods p
      ON fcac.lyft_id = p.lyft_id
      AND p.period_id = '1'
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
             34

)

, RCS AS (

  SELECT db.*,
  
         --RCS ANY REASON
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.consequence_level = 3 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_3_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) < DATE_TRUNC('day',db.end_date_time)                                                                                     AND rcs.consequence_level = 3 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_3_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 3 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_3_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 3 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_3_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 3 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_3_prior_1wks,
         
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND db.end_date_time + interval '45' day                  AND rcs.consequence_level = 2 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_2_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) < DATE_TRUNC('day',db.end_date_time)                                                                                     AND rcs.consequence_level = 2 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_2_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 2 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_2_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 2 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_2_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 2 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_2_prior_1wks,
         
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.consequence_level = 1 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_1_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) < DATE_TRUNC('day',db.end_date_time)                                                                                     AND rcs.consequence_level = 1 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_1_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 1 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_1_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 1 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_1_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day AND rcs.consequence_level = 1 AND rcs.consequence_level_increase = 1 THEN 1 ELSE 0 END) AS num_rcs_level_1_prior_1wks,
         
         
         --FAILED CHARGES
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.failed_charge_level = 3 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_3_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.failed_charge_level = 3 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_3_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 3 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_3_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 3 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_3_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 3 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_3_prior_1wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.failed_charge_level = 2 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_2_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.failed_charge_level = 2 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_2_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 2 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_2_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 2 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_2_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 2 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_2_prior_1wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.failed_charge_level = 1 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_1_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.failed_charge_level = 1 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_1_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 1 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_1_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 1 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_1_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.failed_charge_level = 1 AND rcs.failed_charge_level_increase = 1 THEN 1 ELSE 0 END) AS num_failed_charge_level_1_prior_1wks,         


          --RIDE COUNT
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.ride_count_requirement_level = 3 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_3_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.ride_count_requirement_level = 3 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_3_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 3 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_3_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 3 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_3_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 3 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_3_prior_1wks,         
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.ride_count_requirement_level = 2 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_2_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.ride_count_requirement_level = 2 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_2_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 2 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_2_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 2 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_2_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 2 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_2_prior_1wks,   
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.ride_count_requirement_level = 1 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_1_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.ride_count_requirement_level = 1 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_1_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 1 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_1_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 1 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_1_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.ride_count_requirement_level = 1 AND rcs.ride_count_requirement_level_increase = 1 THEN 1 ELSE 0 END) AS num_ride_count_level_1_prior_1wks,


        --OVERDUE RENTAL
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.overdue_level = 3 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_3_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.overdue_level = 3 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_3_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 3 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_3_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 3 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_3_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 3 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_3_prior_1wks,         
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.overdue_level = 2 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_2_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.overdue_level = 2 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_2_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 2 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_2_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 2 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_2_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 2 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_2_prior_1wks,   
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day  AND DATE_TRUNC('day',db.end_date_time) + interval '45' day AND rcs.overdue_level = 1 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_1_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                AND rcs.overdue_level = 1 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_1_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 1 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_1_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 1 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_1_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day  AND rcs.overdue_level = 1 AND rcs.overdue_level_increase = 1 THEN 1 ELSE 0 END) AS num_overdue_level_1_prior_1wks,


        --PREVENTATIVE MAINTENANCE
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day AND DATE_TRUNC('day',db.end_date_time) + interval '45' day                  AND rcs.pm_level = 3 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_3_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                                AND rcs.pm_level = 3 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_3_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 3 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_3_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 3 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_3_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 3 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_3_prior_1wks,         
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day AND DATE_TRUNC('day',db.end_date_time) + interval '45' day                  AND rcs.pm_level = 2 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_2_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                                AND rcs.pm_level = 2 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_2_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 2 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_2_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 2 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_2_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 2 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_2_prior_1wks,   
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) + interval '8' day AND DATE_TRUNC('day',db.end_date_time) + interval '45' day                  AND rcs.pm_level = 1 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_1_45d_post_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at)       < DATE_TRUNC('day',db.end_date_time)                                                                                                AND rcs.pm_level = 1 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_1_pre_rental,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '21' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 1 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_1_prior_3wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '14' day AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 1 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_1_prior_2wks,
         SUM(CASE WHEN DATE_TRUNC('day',rcs.occurred_at) BETWEEN DATE_TRUNC('day',db.end_date_time) - interval '7' day  AND DATE_TRUNC('day',db.end_date_time) - interval '1' day                  AND rcs.pm_level = 1 AND rcs.pm_level_increase = 1 THEN 1 ELSE 0 END) AS num_pm_level_1_prior_1wks

    FROM driving_behavior db
    LEFT JOIN hive.public.rcs_increases  rcs
      ON db.lyft_id = rcs.lyft_id
   GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
            34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49

)

, fees AS (

  SELECT rcs.*,
         SUM(CASE WHEN DATE_TRUNC('day',fees.fee_issue_date) BETWEEN DATE_TRUNC('day',rcs.end_date_time) - interval '28' day AND DATE_TRUNC('day',rcs.end_date_time) - interval '1' day THEN amount*0.01 ELSE 0.0 END) AS fees_prior_4wks,
         SUM(CASE WHEN DATE_TRUNC('day',fees.fee_issue_date)       < DATE_TRUNC('day',rcs.end_date_time) THEN amount*0.01 ELSE 0.0 END)*1.0/rcs.days_active_pre_rental AS fees_per_day_active
    FROM rcs
    LEFT JOIN hive.default.event_rentals_provider_fee fees
      ON CAST(fees.user_id AS BIGINT) = rcs.lyft_id
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
             34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
             64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,
             94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,
             118,119,120,121,122,123,124

)

SELECT fees.*,
      
       --FEE AMOUNTS
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                                                        THEN amount ELSE 0 END) AS sum_traffic_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day                                        THEN amount ELSE 0 END) AS sum_traffic_violation_fees_prior_3wks,
       
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'PARKING'               THEN amount ELSE 0 END) AS sum_parking_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'PARKING'               THEN amount ELSE 0 END) AS sum_parking_violation_fees_prior_3wks,
       
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'TOLLS'                 THEN amount ELSE 0 END) AS sum_tolls_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'TOLLS'                 THEN amount ELSE 0 END) AS sum_tolls_violation_fees_prior_3wks,
              
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'RED LIGHT'             THEN amount ELSE 0 END) AS sum_redlight_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'RED LIGHT'             THEN amount ELSE 0 END) AS sum_redlight_violation_fees_prior_3wks,
              
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'SPEEDING'              THEN amount ELSE 0 END) AS sum_speeding_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'SPEEDING'              THEN amount ELSE 0 END) AS sum_speeding_violation_fees_prior_3wks,
       
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'IMPROPER REGISTRATION' THEN amount ELSE 0 END) AS sum_improperregistration_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'IMPROPER REGISTRATION' THEN amount ELSE 0 END) AS sum_improperregistration_violation_fees_prior_3wks,

       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'MISCELLANEOUS'         THEN amount ELSE 0 END) AS sum_misc_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'MISCELLANEOUS'         THEN amount ELSE 0 END) AS sum_misc_violation_fees_prior_3wks,
       
       
       --COUNTS
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                                                        THEN 1 ELSE 0 END) AS num_traffic_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day                                        THEN 1 ELSE 0 END) AS num_traffic_violation_fees_prior_3wks,
       
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'PARKING'               THEN 1 ELSE 0 END) AS num_parking_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'PARKING'               THEN 1 ELSE 0 END) AS num_parking_violation_fees_prior_3wks,
       
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'TOLLS'                 THEN 1 ELSE 0 END) AS num_tolls_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'TOLLS'                 THEN 1 ELSE 0 END) AS num_tolls_violation_fees_prior_3wks,
              
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'RED LIGHT'             THEN 1 ELSE 0 END) AS num_redlight_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'RED LIGHT'             THEN 1 ELSE 0 END) AS num_redlight_violation_fees_prior_3wks,
              
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'SPEEDING'              THEN 1 ELSE 0 END) AS num_speeding_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'SPEEDING'              THEN 1 ELSE 0 END) AS num_speeding_violation_fees_prior_3wks,
       
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'IMPROPER REGISTRATION' THEN 1 ELSE 0 END) AS num_improperregistration_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'IMPROPER REGISTRATION' THEN 1 ELSE 0 END) AS num_improperregistration_violation_fees_prior_3wks,

       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at)       < DATE_TRUNC('day',fees.end_date_time)                                                                                 AND category = 'MISCELLANEOUS'         THEN 1 ELSE 0 END) AS num_misc_violation_fees_pre_rental,
       SUM(CASE WHEN DATE_TRUNC('day',rtvc.occurred_at) BETWEEN DATE_TRUNC('day',fees.end_date_time) - interval '28' day AND DATE_TRUNC('day',fees.end_date_time) - interval '1' day AND category = 'MISCELLANEOUS'         THEN 1 ELSE 0 END) AS num_misc_violation_fees_prior_3wks
       
  FROM fees
  LEFT JOIN hive.public.rental_traffic_violation_charges rtvc
    ON rtvc.lyft_id = fees.lyft_id
 GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
          34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,
          64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,
          94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,
          118,119,120,121,122,123,124,125,126


"""


df = presto.query(sql)

In [3]:
# Number of Rental Weeks in Training and Validaiton Dataset
df.shape

(1328134, 154)

In [4]:
# Number of renters in the Training and Validation Dataset
df.lyft_id.nunique()

113024

In [5]:
for column in df.columns:
    print("'" +str(column)+"',")

'lyft_id',
'region',
'rental_id',
'rental_provider_id',
'start_date_time',
'end_date_time',
'num_failed_charges_45d_post_rental',
'sum_failed_charges_45d_post_rental',
'num_failed_charges_pre_rental',
'num_failed_charges_prior_3wks',
'num_failed_charges_prior_2wks',
'num_failed_charges_prior_1wk',
'sum_failed_charges_pre_rental',
'sum_failed_charges_prior_3wks',
'sum_failed_charges_prior_2wks',
'sum_failed_charges_prior_1wk',
'num_init_failed_charges_pre_rental',
'num_init_failed_charges_prior_3wks',
'num_init_failed_charges_prior_2wks',
'num_init_failed_charges_prior_1wk',
'sum_init_failed_charges_pre_rental',
'sum_init_failed_charges_prior_3wks',
'sum_init_failed_charges_prior_2wks',
'sum_init_failed_charges_prior_1wk',
'num_chargebacks_45d_post_rental',
'sum_chargebacks_45d_post_rental',
'num_chargebacks_pre_rental',
'num_chargebacks_prior_3wks',
'num_chargebacks_prior_2wks',
'num_chargebacks_prior_1wk',
'sum_chargebacks_pre_rental',
'sum_chargebacks_prior_3wks',
'sum_chargebacks_pr

########VERSION 1


# Isolating features I'd like to use in the model

columns_for_training =[ 
'lyft_id',
#'region',
#'rental_id',
#'rental_provider_id',
#'start_date_time',
'end_date_time',
#'num_failed_charges_45d_post_rental',
'sum_failed_charges_45d_post_rental',
'num_failed_charges_pre_rental',
'num_failed_charges_prior_3wks',
'num_failed_charges_prior_2wks',
'num_failed_charges_prior_1wk',
'sum_failed_charges_pre_rental',
'sum_failed_charges_prior_3wks',
'sum_failed_charges_prior_2wks',
'sum_failed_charges_prior_1wk',
'num_init_failed_charges_pre_rental',
'num_init_failed_charges_prior_3wks',
'num_init_failed_charges_prior_2wks',
'num_init_failed_charges_prior_1wk',
'sum_init_failed_charges_pre_rental',
'sum_init_failed_charges_prior_3wks',
'sum_init_failed_charges_prior_2wks',
'sum_init_failed_charges_prior_1wk',
#'num_chargebacks_45d_post_rental',
#'sum_chargebacks_45d_post_rental',
'num_chargebacks_pre_rental',
'num_chargebacks_prior_3wks',
'num_chargebacks_prior_2wks',
'num_chargebacks_prior_1wk',
'sum_chargebacks_pre_rental',
'sum_chargebacks_prior_3wks',
'sum_chargebacks_prior_2wks',
'sum_chargebacks_prior_1wk',
#'rides_45d_post_rental',
'rides_pre_rental',
'rides_prior_3wks',
'rides_prior_2wks',
'rides_prior_1wks',
#'hours_45d_post_rental',
#'hours_pre_rental',
#'hours_prior_3wks',
#'hours_prior_2wks',
#'hours_prior_1wks',
#'days_active_45d_post_rental',
'days_active_pre_rental',
'days_active_prior_3wks',
'days_active_prior_2wks',
'days_active_prior_1wks',
#'num_rcs_level_3_45d_post_rental',
'num_rcs_level_3_pre_rental',
'num_rcs_level_3_prior_3wks',
'num_rcs_level_3_prior_2wks',
'num_rcs_level_3_prior_1wks',
#'num_rcs_level_2_45d_post_rental',
'num_rcs_level_2_pre_rental',
'num_rcs_level_2_prior_3wks',
'num_rcs_level_2_prior_2wks',
'num_rcs_level_2_prior_1wks',
#'num_rcs_level_1_45d_post_rental',
'num_rcs_level_1_pre_rental',
'num_rcs_level_1_prior_3wks',
'num_rcs_level_1_prior_2wks',
'num_rcs_level_1_prior_1wks',
#'num_failed_charge_level_3_45d_post_rental',
'num_failed_charge_level_3_pre_rental',
'num_failed_charge_level_3_prior_3wks',
'num_failed_charge_level_3_prior_2wks',
'num_failed_charge_level_3_prior_1wks',
#'num_failed_charge_level_2_45d_post_rental',
'num_failed_charge_level_2_pre_rental',
'num_failed_charge_level_2_prior_3wks',
'num_failed_charge_level_2_prior_2wks',
'num_failed_charge_level_2_prior_1wks',
#'num_failed_charge_level_1_45d_post_rental',
'num_failed_charge_level_1_pre_rental',
'num_failed_charge_level_1_prior_3wks',
'num_failed_charge_level_1_prior_2wks',
'num_failed_charge_level_1_prior_1wks',
#'num_ride_count_level_3_45d_post_rental',
'num_ride_count_level_3_pre_rental',
'num_ride_count_level_3_prior_3wks',
'num_ride_count_level_3_prior_2wks',
'num_ride_count_level_3_prior_1wks',
#'num_ride_count_level_2_45d_post_rental',
'num_ride_count_level_2_pre_rental',
'num_ride_count_level_2_prior_3wks',
'num_ride_count_level_2_prior_2wks',
'num_ride_count_level_2_prior_1wks',
#'num_ride_count_level_1_45d_post_rental',
'num_ride_count_level_1_pre_rental',
'num_ride_count_level_1_prior_3wks',
'num_ride_count_level_1_prior_2wks',
'num_ride_count_level_1_prior_1wks',
#'num_overdue_level_3_45d_post_rental',
'num_overdue_level_3_pre_rental',
'num_overdue_level_3_prior_3wks',
'num_overdue_level_3_prior_2wks',
'num_overdue_level_3_prior_1wks',
#'num_overdue_level_2_45d_post_rental',
'num_overdue_level_2_pre_rental',
'num_overdue_level_2_prior_3wks',
'num_overdue_level_2_prior_2wks',
'num_overdue_level_2_prior_1wks',
#'num_overdue_level_1_45d_post_rental',
'num_overdue_level_1_pre_rental',
'num_overdue_level_1_prior_3wks',
'num_overdue_level_1_prior_2wks',
'num_overdue_level_1_prior_1wks',
#'num_pm_level_3_45d_post_rental',
'num_pm_level_3_pre_rental',
'num_pm_level_3_prior_3wks',
'num_pm_level_3_prior_2wks',
'num_pm_level_3_prior_1wks',
#'num_pm_level_2_45d_post_rental',
'num_pm_level_2_pre_rental',
'num_pm_level_2_prior_3wks',
'num_pm_level_2_prior_2wks',
'num_pm_level_2_prior_1wks',
#'num_pm_level_1_45d_post_rental',
'num_pm_level_1_pre_rental',
'num_pm_level_1_prior_3wks',
'num_pm_level_1_prior_2wks',
'num_pm_level_1_prior_1wks',
'fees_prior_4wks',
#'fees_per_day_active',
'sum_traffic_violation_fees_pre_rental',
'sum_traffic_violation_fees_prior_3wks',
'sum_parking_violation_fees_pre_rental',
'sum_parking_violation_fees_prior_3wks',
'sum_tolls_violation_fees_pre_rental',
'sum_tolls_violation_fees_prior_3wks',
'sum_redlight_violation_fees_pre_rental',
'sum_redlight_violation_fees_prior_3wks',
'sum_speeding_violation_fees_pre_rental',
'sum_speeding_violation_fees_prior_3wks',
'sum_improperregistration_violation_fees_pre_rental',
'sum_improperregistration_violation_fees_prior_3wks',
'sum_misc_violation_fees_pre_rental',
'sum_misc_violation_fees_prior_3wks',
'num_traffic_violation_fees_pre_rental',
'num_traffic_violation_fees_prior_3wks',
'num_parking_violation_fees_pre_rental',
'num_parking_violation_fees_prior_3wks',
'num_tolls_violation_fees_pre_rental',
'num_tolls_violation_fees_prior_3wks',
'num_redlight_violation_fees_pre_rental',
'num_redlight_violation_fees_prior_3wks',
'num_speeding_violation_fees_pre_rental',
'num_speeding_violation_fees_prior_3wks',
'num_improperregistration_violation_fees_pre_rental',
'num_improperregistration_violation_fees_prior_3wks',
'num_misc_violation_fees_pre_rental',
'num_misc_violation_fees_prior_3wks']

In [6]:
########VERSION 2


# Isolating features I'd like to use in the model

columns_for_training =[ 
'lyft_id',
#'region',
#'rental_id',
#'rental_provider_id',
#'start_date_time',
'end_date_time',
#'num_failed_charges_45d_post_rental',
'sum_failed_charges_45d_post_rental',
'num_failed_charges_pre_rental',
'num_failed_charges_prior_3wks',
#'num_failed_charges_prior_2wks',
#'num_failed_charges_prior_1wk',
'sum_failed_charges_pre_rental',
'sum_failed_charges_prior_3wks',
#'sum_failed_charges_prior_2wks',
#'sum_failed_charges_prior_1wk',
'num_init_failed_charges_pre_rental',
'num_init_failed_charges_prior_3wks',
#'num_init_failed_charges_prior_2wks',
#'num_init_failed_charges_prior_1wk',
'sum_init_failed_charges_pre_rental',
'sum_init_failed_charges_prior_3wks',
#'sum_init_failed_charges_prior_2wks',
#'sum_init_failed_charges_prior_1wk',
#'num_chargebacks_45d_post_rental',
#'sum_chargebacks_45d_post_rental',
'num_chargebacks_pre_rental',
'num_chargebacks_prior_3wks',
#'num_chargebacks_prior_2wks',
#'num_chargebacks_prior_1wk',
'sum_chargebacks_pre_rental',
'sum_chargebacks_prior_3wks',
#'sum_chargebacks_prior_2wks',
#'sum_chargebacks_prior_1wk',
#'rides_45d_post_rental',
'rides_pre_rental',
'rides_prior_3wks',
#'rides_prior_2wks',
#'rides_prior_1wks',
#'hours_45d_post_rental',
#'hours_pre_rental',
#'hours_prior_3wks',
#'hours_prior_2wks',
#'hours_prior_1wks',
#'days_active_45d_post_rental',
'days_active_pre_rental',
'days_active_prior_3wks',
#'days_active_prior_2wks',
#'days_active_prior_1wks',
#'num_rcs_level_3_45d_post_rental',
'num_rcs_level_3_pre_rental',
'num_rcs_level_3_prior_3wks',
#'num_rcs_level_3_prior_2wks',
#'num_rcs_level_3_prior_1wks',
#'num_rcs_level_2_45d_post_rental',
'num_rcs_level_2_pre_rental',
'num_rcs_level_2_prior_3wks',
#'num_rcs_level_2_prior_2wks',
#'num_rcs_level_2_prior_1wks',
#'num_rcs_level_1_45d_post_rental',
'num_rcs_level_1_pre_rental',
'num_rcs_level_1_prior_3wks',
#'num_rcs_level_1_prior_2wks',
#'num_rcs_level_1_prior_1wks',
#'num_failed_charge_level_3_45d_post_rental',
'num_failed_charge_level_3_pre_rental',
'num_failed_charge_level_3_prior_3wks',
#'num_failed_charge_level_3_prior_2wks',
#'num_failed_charge_level_3_prior_1wks',
#'num_failed_charge_level_2_45d_post_rental',
'num_failed_charge_level_2_pre_rental',
'num_failed_charge_level_2_prior_3wks',
#'num_failed_charge_level_2_prior_2wks',
#'num_failed_charge_level_2_prior_1wks',
#'num_failed_charge_level_1_45d_post_rental',
'num_failed_charge_level_1_pre_rental',
'num_failed_charge_level_1_prior_3wks',
#'num_failed_charge_level_1_prior_2wks',
#'num_failed_charge_level_1_prior_1wks',
#'num_ride_count_level_3_45d_post_rental',
'num_ride_count_level_3_pre_rental',
'num_ride_count_level_3_prior_3wks',
#'num_ride_count_level_3_prior_2wks',
#'num_ride_count_level_3_prior_1wks',
#'num_ride_count_level_2_45d_post_rental',
'num_ride_count_level_2_pre_rental',
'num_ride_count_level_2_prior_3wks',
#'num_ride_count_level_2_prior_2wks',
#'num_ride_count_level_2_prior_1wks',
#'num_ride_count_level_1_45d_post_rental',
'num_ride_count_level_1_pre_rental',
'num_ride_count_level_1_prior_3wks',
#'num_ride_count_level_1_prior_2wks',
#'num_ride_count_level_1_prior_1wks',
#'num_overdue_level_3_45d_post_rental',
'num_overdue_level_3_pre_rental',
'num_overdue_level_3_prior_3wks',
#'num_overdue_level_3_prior_2wks',
#'num_overdue_level_3_prior_1wks',
#'num_overdue_level_2_45d_post_rental',
'num_overdue_level_2_pre_rental',
'num_overdue_level_2_prior_3wks',
#'num_overdue_level_2_prior_2wks',
#'num_overdue_level_2_prior_1wks',
#'num_overdue_level_1_45d_post_rental',
'num_overdue_level_1_pre_rental',
'num_overdue_level_1_prior_3wks',
#'num_overdue_level_1_prior_2wks',
#'num_overdue_level_1_prior_1wks',
#'num_pm_level_3_45d_post_rental',
'num_pm_level_3_pre_rental',
'num_pm_level_3_prior_3wks',
#'num_pm_level_3_prior_2wks',
#'num_pm_level_3_prior_1wks',
#'num_pm_level_2_45d_post_rental',
'num_pm_level_2_pre_rental',
'num_pm_level_2_prior_3wks',
#'num_pm_level_2_prior_2wks',
#'num_pm_level_2_prior_1wks',
#'num_pm_level_1_45d_post_rental',
'num_pm_level_1_pre_rental',
'num_pm_level_1_prior_3wks',
#'num_pm_level_1_prior_2wks',
#'num_pm_level_1_prior_1wks',
'fees_prior_4wks',
#'fees_per_day_active',
'sum_traffic_violation_fees_pre_rental',
'sum_traffic_violation_fees_prior_3wks',
'sum_parking_violation_fees_pre_rental',
'sum_parking_violation_fees_prior_3wks',
'sum_tolls_violation_fees_pre_rental',
'sum_tolls_violation_fees_prior_3wks',
'sum_redlight_violation_fees_pre_rental',
'sum_redlight_violation_fees_prior_3wks',
'sum_speeding_violation_fees_pre_rental',
'sum_speeding_violation_fees_prior_3wks',
'sum_improperregistration_violation_fees_pre_rental',
'sum_improperregistration_violation_fees_prior_3wks',
'sum_misc_violation_fees_pre_rental',
'sum_misc_violation_fees_prior_3wks',
'num_traffic_violation_fees_pre_rental',
'num_traffic_violation_fees_prior_3wks',
'num_parking_violation_fees_pre_rental',
'num_parking_violation_fees_prior_3wks',
'num_tolls_violation_fees_pre_rental',
'num_tolls_violation_fees_prior_3wks',
'num_redlight_violation_fees_pre_rental',
'num_redlight_violation_fees_prior_3wks',
'num_speeding_violation_fees_pre_rental',
'num_speeding_violation_fees_prior_3wks',
'num_improperregistration_violation_fees_pre_rental',
'num_improperregistration_violation_fees_prior_3wks',
'num_misc_violation_fees_pre_rental',
'num_misc_violation_fees_prior_3wks']

In [7]:
# Dropping any features not in the list I want to use
dfr = df[columns_for_training]

In [8]:
# Removing rows with null values
# include NULLs with xgboost
dfr = dfr.dropna(axis = 0)

In [9]:
#Checking for anymore nulls
dfr.isnull().any().any()

False

In [10]:
#Defining the target variable (failed charge y/n)
dfr['y'] = 0
dfr.loc[dfr.sum_failed_charges_45d_post_rental > 0, 'y'] = 1

In [11]:
# What % of rentals have a failed charge in the proceeding 45 days?  Note there will be some duplicates here
dfr.y.mean()

0.05929597465316

# Decision Tree

In [53]:
from sklearn import tree

In [54]:
# Splitting training and testing datasets by drivers, not by rental weeks
# This ensures that the training dataset does not have any information from drivers in the validation set

train_lyft_ids = pd.DataFrame(dfr.lyft_id.unique(), columns = {'lyft_id'}).sample(frac = 0.75)

train = dfr.loc[dfr.lyft_id.isin(train_lyft_ids.lyft_id)]
test = dfr.loc[~dfr.lyft_id.isin(train_lyft_ids.lyft_id)]

In [55]:
# Building the classifier

clf = tree.DecisionTreeClassifier(max_depth = 6)
X_train = train.drop(columns=['lyft_id','end_date_time','sum_failed_charges_45d_post_rental','y'])
clf = clf.fit(X_train,train['y'])

In [56]:
# mean accurarcy on training dataset (% of predictions correct)

clf.score(train.drop(columns=['lyft_id','end_date_time','sum_failed_charges_45d_post_rental','y']),train['y'])


0.9408987395456254

In [57]:
# mean accurarcy on testing dataset (% of predictions correct)

clf.score(test.drop(columns=['lyft_id','end_date_time','sum_failed_charges_45d_post_rental','y']),test['y'])


0.9403054975142301

In [58]:

X_test  = test.drop(columns=['lyft_id','end_date_time','sum_failed_charges_45d_post_rental','y'])
df_test_scores = clf.predict_proba(X_test)
df_test_scores = pd.DataFrame(df_test_scores, columns=['p_no_fc', 'p_fc'])
test.reset_index(inplace = True)
test['p_fc'] = df_test_scores['p_fc']
test['lyft_id'] = test['lyft_id'].astype(str) + '_'
#test.columns
scores_out = test[['lyft_id','end_date_time','p_fc']]


/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [18]:
scores_out.lyft_id.nunique()

28256

In [19]:
scores_out.head()

,lyft_id,end_date_time,p_fc
0,902608454024308602_,2018-09-15 00:17:18,0.298732
1,1176051224947476816_,2019-01-30 16:09:03,0.151433
2,933958503979257188_,2019-03-20 15:50:15,0.008450
3,1048931768034607434_,2018-07-19 17:09:14,0.119891
4,1159547504471490960_,2018-09-11 19:22:09,0.073362


In [20]:
def upload_df(df, tablename, columns_and_dtypes, replace_table=False):
    if replace_table == True:
        print("Dropping existing table.")
        drop_query = 'DROP TABLE IF EXISTS ' + str(tablename)
        print(drop_query)
        presto.query(drop_query)
    
    print("Creating table.")
    create_query = 'CREATE TABLE IF NOT EXISTS ' + tablename + ' (' + columns_and_dtypes + ')'
    print(create_query)
    presto.query(create_query)
    
    print("Uploading dataframe.")
    i = 0
    increment = 10000
    
    current_inc = 1
    num_increments = math.ceil(df.shape[0]/increment)
    
    while i <= df.shape[0]:
        print('Insert ' + str(current_inc) + '/' + str(num_increments))
        df_trunc = df[i:i+increment]
        insert_query = 'INSERT INTO '+ tablename+ ' VALUES'
        for row in range(0,increment):
            insert_row = "("
            try:
                for column in df_trunc.iloc[row]:
                    insert_row += str(column)+','
                insert_row = insert_row[:-1] + "),"
                insert_query += insert_row[:-1] + ','
            except:
                pass
        #print(insert_query[:-1])
        presto.query(insert_query[:-1])
        i += increment
        current_inc += 1
        
    print("Upload Finished!")

In [21]:
df_to_insert = scores_out.copy()

In [22]:
df_to_insert.head()

,lyft_id,end_date_time,p_fc
0,902608454024308602_,2018-09-15 00:17:18,0.298732
1,1176051224947476816_,2019-01-30 16:09:03,0.151433
2,933958503979257188_,2019-03-20 15:50:15,0.008450
3,1048931768034607434_,2018-07-19 17:09:14,0.119891
4,1159547504471490960_,2018-09-11 19:22:09,0.073362


In [23]:
df_to_insert.end_date_time = df_to_insert.end_date_time.astype(str)

In [24]:
df_to_insert.end_date_time = "'"+df_to_insert.end_date_time+"'"
df_to_insert.lyft_id = "'"+df_to_insert.lyft_id+"'"

In [25]:
c = """
lyft_id varchar,
end_date_time varchar,
p_fc double
"""


In [26]:
upload_df(df_to_insert, 'hive.myamane.systematic_dnr_validation_scores', c, replace_table=True)

Dropping existing table.
DROP TABLE IF EXISTS hive.myamane.systematic_dnr_validation_scores
Creating table.
CREATE TABLE IF NOT EXISTS hive.myamane.systematic_dnr_validation_scores (
lyft_id varchar,
end_date_time varchar,
p_fc double
)
Uploading dataframe.
Insert 1/34
Insert 2/34
Insert 3/34
Insert 4/34
Insert 5/34
Insert 6/34
Insert 7/34
Insert 8/34
Insert 9/34
Insert 10/34
Insert 11/34
Insert 12/34
Insert 13/34
Insert 14/34
Insert 15/34
Insert 16/34
Insert 17/34
Insert 18/34
Insert 19/34
Insert 20/34
Insert 21/34
Insert 22/34
Insert 23/34
Insert 24/34
Insert 25/34
Insert 26/34
Insert 27/34
Insert 28/34
Insert 29/34
Insert 30/34
Insert 31/34
Insert 32/34
Insert 33/34
Insert 34/34
Upload Finished!


In [27]:
# Iterating through different threshold to show what the tradeoff is at different p_fc_thresholds


# This is the df that will be filled with scores
df_out = pd.DataFrame({'pfc_threshold':[-1],
             'num_dvrs': [0],
             'avg_pfc_score': [0],
             'num_fc_prevented': [0],
             'sum_fc_prevented': [0],
             'num_rides_foregone': [0],
             'num_rentals_foregone': [0],
             'fc_per_ride_tradeoff': [0],
             'fc_per_rental_tradeoff': [0]
             })

# Iterating through different scores
for p_fc_threshold in np.linspace(0,0.5,11):
    
    print("Running query for p_fc_threshold: "+str(p_fc_threshold))
    
    ## Query built and validated here: https://app.mode.com/editor/lyft/reports/421871a78f85
    
    validation_query = f"""

    WITH scored_rental_weeks AS (
    SELECT CAST(REPLACE(lyft_id,'_') AS BIGINT) as lyft_id,
           CAST(end_date_time AS TIMESTAMP) as end_date_time,
           p_fc
      FROM hive.myamane.systematic_dnr_validation_scores
    )

    , rentals_to_dnr AS (
      SELECT lyft_id,
             MIN(end_date_time) as end_date_time,
             MIN_BY(p_fc, end_date_time) as p_fc_score_at_rental_end
        FROM scored_rental_weeks
        WHERE p_fc >= {p_fc_threshold}
      GROUP BY 1
    )

    , fc_prevented AS (
      SELECT dnr.lyft_id,
             dnr.end_date_time as date_dnr,
             dnr.p_fc_score_at_rental_end,
             SUM(CASE WHEN fgc.created_at > dnr.end_date_time + interval '8' day AND (fgc.succeeded_at IS NULL OR fgc.succeeded_at > fgc.created_at + interval '45' day) THEN 1 ELSE 0 END) as number_failed_charges_prevented,
             SUM(CASE WHEN fgc.created_at > dnr.end_date_time + interval '8' day AND (fgc.succeeded_at IS NULL OR fgc.succeeded_at > fgc.created_at + interval '45' day) THEN fgc.amount*0.01 ELSE 0.0 END) as amount_failed_charges_prevented
        FROM rentals_to_dnr dnr
        LEFT JOIN hive.redshift.fact_green_charges fgc
          ON dnr.lyft_id = fgc.lyft_id
         AND fgc.order_type IN('rental_deposit','rentalSurcharge')
         AND fgc.status<>'forgiven'
       GROUP BY 1,2,3
     )
     
     , daily_xd_rides AS (
     
      SELECT rdrp.*
        FROM hive.core.rollup_driver_region_periods rdrp
        JOIN hive.redshift.dimension_rentals dr
          ON dr.lyft_id = rdrp.lyft_id
         AND rdrp.time_id BETWEEN DATE_TRUNC('day',dr.start_date_time) AND DATE_TRUNC('day',COALESCE(dr.end_date_time,dr.next_start_date_time, current_date))
         AND dr.start_date_time + interval '90' day > COALESCE(dr.end_date_time,dr.next_start_date_time, current_date)
        WHERE rdrp.period_id = '1'
     
     )

    , rides_foregone AS (
      SELECT fp.*,
             SUM(CASE WHEN rdrp.time_id > fp.date_dnr THEN rdrp.rides_completed ELSE 0 END) AS rides_forgone
        FROM fc_prevented fp
        LEFT JOIN daily_xd_rides rdrp
          ON fp.lyft_id = rdrp.lyft_id
       GROUP BY 1,2,3,4,5
    )

    , rentals_foregone AS (
      SELECT rf.*,
             SUM(CASE WHEN DATE_TRUNC('day',dr.start_date_time) >= DATE_TRUNC('day',rf.date_dnr) THEN 1 ELSE 0 END) AS num_rentals_foregone   
        FROM rides_foregone rf
        LEFT JOIN hive.redshift.dimension_rentals dr
          ON dr.lyft_id = rf.lyft_id
       GROUP BY 1,2,3,4,5,6 
    )
      SELECT * 
        FROM rentals_foregone 

    """

    print("Query finished for p_fc_threshold: "+str(p_fc_threshold))
    
    df_validation = presto.query(validation_query)

    df_out = df_out.append(pd.DataFrame({'pfc_threshold':[p_fc_threshold],
                 'num_dvrs': [df_validation.lyft_id.count()],
                 'avg_pfc_score': [df_validation.p_fc_score_at_rental_end.mean()],
                 'num_fc_prevented': [df_validation.number_failed_charges_prevented.sum()],
                 'sum_fc_prevented': [df_validation.amount_failed_charges_prevented.sum()],
                 'num_rides_foregone': [df_validation.rides_forgone.sum()],
                 'num_rentals_foregone': [df_validation.num_rentals_foregone.sum()],
                 'fc_per_ride_tradeoff': [float(df_validation.amount_failed_charges_prevented.sum())/df_validation.rides_forgone.sum()],
                 'fc_per_rental_tradeoff': [float(df_validation.amount_failed_charges_prevented.sum())/df_validation.num_rentals_foregone.sum()]
                 }))
    
print("Finished!")        



Running query for p_fc_threshold: 0.0
Query finished for p_fc_threshold: 0.0
Running query for p_fc_threshold: 0.05
Query finished for p_fc_threshold: 0.05
Running query for p_fc_threshold: 0.1
Query finished for p_fc_threshold: 0.1
Running query for p_fc_threshold: 0.15000000000000002
Query finished for p_fc_threshold: 0.15000000000000002
Running query for p_fc_threshold: 0.2
Query finished for p_fc_threshold: 0.2
Running query for p_fc_threshold: 0.25
Query finished for p_fc_threshold: 0.25
Running query for p_fc_threshold: 0.30000000000000004
Query finished for p_fc_threshold: 0.30000000000000004
Running query for p_fc_threshold: 0.35000000000000003
Query finished for p_fc_threshold: 0.35000000000000003
Running query for p_fc_threshold: 0.4
Query finished for p_fc_threshold: 0.4
Running query for p_fc_threshold: 0.45
Query finished for p_fc_threshold: 0.45
Running query for p_fc_threshold: 0.5
Query finished for p_fc_threshold: 0.5
Finished!


In [28]:
## Calculating the total failed charges, number rides, and rentals for the drivers in our validation dataset
## This allows us to calculate the % of failed charges and % rides/rentals we are cutting out at different p_fc_thresholds

total_metrics_query = f"""
WITH scored_dvrs AS (
    SELECT DISTINCT CAST(REPLACE(lyft_id,'_') AS BIGINT) as lyft_id
      FROM hive.myamane.systematic_dnr_validation_scores
    )

    , total_fc AS (
      SELECT dnr.lyft_id,
             SUM(CASE WHEN fgc.created_at > timestamp '2018-06-01' AND (fgc.succeeded_at IS NULL OR fgc.succeeded_at > fgc.created_at + interval '45' day) THEN 1 ELSE 0 END) as total_number_failed_charges,
             SUM(CASE WHEN fgc.created_at > timestamp '2018-06-01' AND (fgc.succeeded_at IS NULL OR fgc.succeeded_at > fgc.created_at + interval '45' day) THEN fgc.amount*0.01 ELSE 0.0 END) as total_amount_failed_charges
        FROM scored_dvrs dnr
        LEFT JOIN hive.redshift.fact_green_charges fgc
          ON dnr.lyft_id = fgc.lyft_id
         AND fgc.order_type IN('rental_deposit','rentalSurcharge')
         AND fgc.status<>'forgiven'
       GROUP BY 1
     )
     
     , daily_xd_rides AS (
     
      SELECT rdrp.*
        FROM hive.core.rollup_driver_region_periods rdrp
        JOIN hive.redshift.dimension_rentals dr
          ON dr.lyft_id = rdrp.lyft_id
         AND rdrp.time_id BETWEEN DATE_TRUNC('day',dr.start_date_time) AND DATE_TRUNC('day',COALESCE(dr.end_date_time,dr.next_start_date_time, current_date))
         AND dr.start_date_time + interval '90' day > COALESCE(dr.end_date_time,dr.next_start_date_time, current_date)
        WHERE rdrp.period_id = '1'
     
     )

    , total_rides AS (
      SELECT tf.*,
             SUM(CASE WHEN rdrp.time_id > timestamp '2018-06-01' THEN rdrp.rides_completed ELSE 0 END) AS total_rides
        FROM total_fc tf
        LEFT JOIN daily_xd_rides rdrp
          ON tf.lyft_id = rdrp.lyft_id
       GROUP BY 1,2,3
    )

    , total_rentals AS (
      SELECT tr.*,
             SUM(CASE WHEN DATE_TRUNC('day',dr.start_date_time) >= timestamp '2018-06-01' THEN 1 ELSE 0 END) AS total_rentals   
        FROM total_rides tr
        LEFT JOIN hive.redshift.dimension_rentals dr
          ON dr.lyft_id = tr.lyft_id
       GROUP BY 1,2,3,4
    )

      SELECT COUNT(lyft_id) as total_num_renters,
             SUM(total_number_failed_charges) as total_number_failed_charges,
             SUM(total_amount_failed_charges) as total_amount_failed_charges,
             SUM(total_rides) as total_rides,
             SUM(total_rentals) as total_rentals
        FROM total_rentals 
"""

df_total_metrics = presto.query(total_metrics_query)


In [29]:
df_total_metrics

,total_num_renters,total_number_failed_charges,total_amount_failed_charges,total_rides,total_rentals
0,28256,25753,6293100.45,31708378,483760


In [30]:
df_out['total_num_renters'] = df_total_metrics['total_num_renters'][0]
df_out['total_number_failed_charges'] = df_total_metrics['total_number_failed_charges'][0]
df_out['total_amount_failed_charges'] = df_total_metrics['total_amount_failed_charges'][0]
df_out['total_rides'] = df_total_metrics['total_rides'][0]
df_out['total_rentals'] = df_total_metrics['total_rentals'][0]

df_out['perc_dvrs_dnrd'] = df_out['num_dvrs']/df_out['total_num_renters']
df_out['perc_num_fc_prevented'] = df_out['num_fc_prevented']/df_out['total_number_failed_charges']
df_out['perc_amt_fc_prevented'] = df_out['sum_fc_prevented']/df_out['total_amount_failed_charges']
df_out['perc_rides_foregone'] = df_out['num_rides_foregone']/df_out['total_rides']
df_out['perc_rentals_foregone'] = df_out['num_rentals_foregone']/df_out['total_rentals']


In [31]:
df_out

,pfc_threshold,num_dvrs,avg_pfc_score,num_fc_prevented,sum_fc_prevented,num_rides_foregone,num_rentals_foregone,fc_per_ride_tradeoff,fc_per_rental_tradeoff,total_num_renters,total_number_failed_charges,total_amount_failed_charges,total_rides,total_rentals,perc_dvrs_dnrd,perc_num_fc_prevented,perc_amt_fc_prevented,perc_rides_foregone,perc_rentals_foregone
0,-1.00,0,0.000000,0,0.00,0,0,0.000000,0.000000,28256,25753,6293100.45,31708378,483760,0.000000,0.000000,0.000000,0.000000,0.000000
0,0.00,28256,0.095843,21531,5275543.23,30158915,461834,0.174925,11.423029,28256,25753,6293100.45,31708378,483760,1.000000,0.836058,0.838306,0.951134,0.954676
0,0.05,21296,0.125802,19470,4797603.78,15757235,286714,0.304470,16.733064,28256,25753,6293100.45,31708378,483760,0.753681,0.756028,0.762359,0.496942,0.592678
0,0.10,15968,0.150876,16934,4186463.15,8481332,183335,0.493609,22.835046,28256,25753,6293100.45,31708378,483760,0.565119,0.657554,0.665247,0.267479,0.378979
0,0.15,9501,0.196496,12186,3041448.81,3660722,93681,0.830833,32.466016,28256,25753,6293100.45,31708378,483760,0.336247,0.473188,0.483299,0.115450,0.193652
0,0.20,5302,0.263393,8109,2047368.85,1641533,44974,1.247230,45.523388,28256,25753,6293100.45,31708378,483760,0.187642,0.314876,0.325335,0.051770,0.092968
0,0.25,3479,0.292034,5574,1405222.78,909692,26006,1.544724,54.034560,28256,25753,6293100.45,31708378,483760,0.123124,0.216441,0.223296,0.028689,0.053758
0,0.30,1193,0.372495,1929,483595.02,240412,7338,2.011526,65.902837,28256,25753,6293100.45,31708378,483760,0.042221,0.074904,0.076845,0.007582,0.015169
0,0.35,1051,0.380633,1811,461856.85,216180,6587,2.136446,70.116419,28256,25753,6293100.45,31708378,483760,0.037196,0.070322,0.073391,0.006818,0.013616
0,0.40,154,0.526277,290,80338.73,51813,1428,1.550552,56.259615,28256,25753,6293100.45,31708378,483760,0.005450,0.011261,0.012766,0.001634,0.002952


In [59]:
from sklearn.metrics import roc_auc_score
y_true = test['y']
y_scores = test['p_fc']
roc_auc_score(y_true, y_scores)

0.7746264654535406

In [66]:
from sklearn.metrics import confusion_matrix
pfc_thresh = 0.1
y_pred = y_scores.copy()
y_pred.loc[y_pred >= pfc_thresh] = 1
y_pred.loc[y_pred < pfc_thresh] = 0
confusion_matrix(y_true, y_pred)

array([[259667,  53553],
       [  9184,  10692]])

## Saving Model

In [40]:
import pickle
filename = 'post_rental_fc_model_100319.sav'
pickle.dump(clf, open(filename, 'wb'))